In [ ]:
#@title *Prepare* { display-mode: "form", run: "auto" }
#@markdown # **Prepare the environment**
mount_gdrive = True #@param {type:"boolean"}
force_remount = False #@param {type:"boolean"}
!rm -rf error_log.txt > /dev/null
import os
import subprocess
from google.colab import drive
if mount_gdrive:
    print("Mounting your Google Drive | Waiting user Allow Access | ", end='')
    try:
        drive.mount('/content/drive/', force_remount=force_remount)
    except Exception as e:
        print(f"[✗]: {e}")

print("Ensuring the LeGen code is existing and updated...", end='')
repo_url = "https://github.com/matheusbach/legen.git"
local_folder = "/content/src"
os.makedirs(local_folder, exist_ok=True)
git_task = "git fetch"
process = subprocess.Popen(git_task, cwd=local_folder, shell=True)
return_code = process.wait()
if return_code == 0:
  git_task = "git fetch && git reset --hard origin/main && git pull"
else:
  !rm -rf local_folder > /dev/null
  os.makedirs(local_folder, exist_ok=True)
  git_task = f"git clone {repo_url} {local_folder}"
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen(git_task, cwd=local_folder, shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

print("Installing or updating pip requirements...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('pip3 install --upgrade -r requirements.txt', cwd=local_folder, shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

print("Install libcudnn8...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('sudo apt install -y libcudnn8', cwd=local_folder, shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

print("Installing FFmpeg and xvfb...", end='')
with open('/content/error_log.txt', 'a') as f:
    process = subprocess.Popen('apt update -y ; apt install ffmpeg xvfb -y', shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

print("Installing fonts...", end='')
with open('/content/error_log.txt', ' 'a') as f:
    process = subprocess.Popen('echo ttf-mscorefonts-installer msttcorefonts/accepted-mscorefonts-eula select true | debconf-set-selections && apt install -y ttf-mscorefonts-installer && fc-cache -fv', shell=True, stderr=f)
    return_code = process.wait()
    print("[✔]" if return_code == 0 else "[✗]")

os.system('Xvfb :1 -screen 0 2560x1440x8  &')
os.environ['DISPLAY'] = ':1.0'
print("\nPreparation tasks done.")

In [ ]:
#@title *Configure* { display-mode: "form", run: "auto" }
#@markdown # **Define Software Settings**
#@markdown Set where your files are located (your Drive is the base /content/drive/MyDrive):
import os
input_path = "/content/drive/MyDrive/LeGen/media" #@param {type:"string"}
#@markdown Set where encoded video/audio files will be saved:
output_dir = "/content/drive/MyDrive/LeGen/encoded" #@param {type:"string"}
#@markdown Overwrite output files if they exist?
overwrite = False #@param {type:"boolean"}
#@markdown Copy non-video/audio files to output?
copy_extra_files = True #@param {type:"boolean"}
#@markdown Maximum parallel encodes (T4: 2-3 is optimal):
max_workers = 2 #@param {type:"integer"}


In [ ]:
#@title *Run* { display-mode: "form" }
#@markdown # **Run Video Encoder**
print("Starting LeGen Video Encoder...")
os.makedirs(input_path, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

query = f" -i '{input_path}' --output_dir '{output_dir}'"
query += " --overwrite" if overwrite else ""
query += " --copy_files" if copy_extra_files else ""
query += f" --max_workers {max_workers}"
print(f"command line: python3 /content/src/legen.py {query}\n")
!python3 /content/src/legen.py $query